## 04 Model Training

#### Import relevant libraries

#### Load model training data

#### Split into test and train sets

#### Scale predictor variables

#### Fit regression model

(a) Linear regression

(b) Random forest

(c) Neural Network

#### Pickle model_fit file(s)